In [202]:
from pyspark.mllib.recommendation import ALS
from numpy import array
import math

In [210]:
songs_rating = sc.textFile('file:/vagrant/triplets_file.txt')

In [212]:
songs_rating=songs_rating.map(lambda line: line.split("\t")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [213]:
songs_rating.take(1)

[(u'b80344d063b5ccb3212f76538f3d9e43d87dca9e', u'SOAKIMP12A8C130995', u'1')]

In [214]:
songs_metadata = sc.textFile('file:/vagrant/song_data.csv')

In [215]:
songs_metadata = songs_metadata.filter(lambda line: line!=songs_metadata_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2],tokens[3],tokens[4])).cache()

In [216]:
songs_metadata.take(1)

[(u'SOQMMHC12AB0180CB8',
  u'"Silent Night"',
  u'"Monster Ballads X-Mas"',
  u'"Faster Pussy cat"',
  u'2003')]

In [217]:
songs_rating_test=songs_rating.map(lambda x: (x[0],(x[1],x[2]))).cache()

In [219]:
user_mapping=songs_rating.map(lambda x: x[0]).distinct().zipWithIndex().cache()

In [220]:
user_mapping.take(1)

[(u'85c1f87fea955d09b4bec2e36aee110927aedf9a', 0)]

In [221]:
songs_rating_1=songs_rating_test.join(user_mapping)

In [222]:
song_rating_user_encoded=songs_rating_1.map(lambda x: (x[1][1],x[1][0][0],x[1][0][1])).cache()

In [223]:
song_rating_user_encoded.take(1)

[(1386, u'SOAFTRR12AF72A8D4D', u'12')]

In [226]:
song_mapping_1=songs_metadata.map(lambda x: x[0]).distinct().zipWithIndex().cache()

In [227]:
song_mapping_1.take(1)

[(u'SOCCXFA12AB0184578', 0)]

In [228]:
song_mapping=songs_metadata.map(lambda x: (x[0],(x[1],x[3],x[4]))).cache()

In [229]:
song_mapping.take(1)

[(u'SOQMMHC12AB0180CB8', (u'"Silent Night"', u'"Faster Pussy cat"', u'2003'))]

In [230]:
songs_mapping_final=song_mapping.join(song_mapping_1).cache()

In [231]:
songs_mapping_final.take(1)

[(u'SOBMRLP12AB0185229',
  ((u'"Geh nicht an mir vorbei"', u'"Christian Anders"', u'0'), 165680))]

In [232]:
songs_mapping_final=song_mapping.join(song_mapping_1).cache()

In [233]:
song_mapping_2=songs_mapping_final.map(lambda x: (x[1][1],x[1][0][0],x[1][0][1],x[1][0][2])).cache()

In [234]:
song_mapping_2.take(1)

[(165680, u'"Geh nicht an mir vorbei"', u'"Christian Anders"', u'0')]

In [240]:
song_rating_user_encoded=song_rating_user_encoded.map(lambda x: (x[1],(x[0],x[2]))).cache()

In [241]:
song_rating_user_encoded.take(1)

[(u'SOAFTRR12AF72A8D4D', (1386, u'12'))]

In [235]:
songs_rating_test_2=song_rating_user_encoded.map(lambda x: (x[1],(x[0],x[2])))

In [242]:
song_data_final=song_rating_user_encoded.join(song_mapping_1).map(lambda x: (x[1][0][0],x[1][1],x[1][0][1])).cache()

In [243]:
song_data_final.take(1)

[(22, 332989, u'1')]

In [244]:
training_RDD, validation_RDD, test_RDD = song_data_final.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [245]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
alpha = 0.05
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.trainImplicit(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter,alpha=alpha)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 6.40947305841
For rank 8 the RMSE is 6.40783451635
For rank 12 the RMSE is 6.40680713064
The best model was trained with rank 12


In [252]:
predictions.take(3)

[((168, 593814), -2.18500074916796e-06),
 ((129, 606250), 0.03730703343618633),
 ((1119, 390392), 0.011713565768778699)]

In [16]:
#rates_and_preds.take(3)

In [246]:
model = ALS.trainImplicit(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter,alpha=alpha)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

For testing data the RMSE is 12.7148452306


In [247]:
from pyspark.mllib.recommendation import MatrixFactorizationModel

model_path = os.path.join('file:/vagrant', 'model', 'songs')

# Save and load model
#model.save(sc, model_path)
same_model = MatrixFactorizationModel.load(sc, model_path)

Py4JJavaError: An error occurred while calling o2619.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/vagrant/model/songs/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1177)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1154)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1154)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:150)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:111)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:316)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1154)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(Pair

In [285]:
songs_rating_counts_RDD = song_data_final.map(lambda x: (x[1], int(x[2]))).reduceByKey(lambda x,y:x+y)

In [301]:
user_id = user_mapping.filter(lambda x: x[0] == 'ff4322e94814d3c7895d07e6f94139b092862611').map(lambda x: x[1]).take(1)[0]

In [302]:
user_id

42

In [303]:
user_id_songs_listened = song_data_final.filter(lambda x: x[0] == user_id)
user_id_songs_listened.take(1)

[(42, 30318, u'1')]

In [304]:
songs_listened_by_user = user_id_songs_listened.map(lambda x: x[1])
songs_listened_by_user_1=[x for x in songs_listened_by_user.toLocalIterator()]

In [307]:
songs_listened_by_user_1

[30318, 400864, 94646, 716236, 230155, 694622, 848895, 840960, 433944, 467996]

In [308]:
user_not_listened_songs_RDD = song_mapping_2.filter(lambda x: x[0] not in songs_listened_by_user_1)\
.map(lambda x: (user_id, x[0]))
#new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_songs_RDD)

In [311]:
user_recommendations_RDD = model.predictAll(user_not_listened_songs_RDD)

In [314]:
user_recommendations_song_RDD = user_recommendations_RDD.map(lambda x: (x.product, x.rating))

In [315]:
user_recommendations_song_RDD.take(1)

[(542894, -0.00018259474369474663)]

In [317]:
song_titles = song_mapping_2.map(lambda x: (int(x[0]),(x[1] + ': ' + x[2] + '-' + x[3])))
#song_titles.take(1)

In [319]:
song_titles.take(4)

[(165680, u'"Geh nicht an mir vorbei": "Christian Anders"-0'),
 (0, u'"Uptown babies": "Max Romeo"-2010'),
 (1, u'"Rock and Roll": "Illaj & Mikey Vegaz"-0'),
 (263389, u'"Italian Movie Theme": Pylon-0')]

In [320]:
user_recommendations_rating_title_and_count_RDD = \
    user_recommendations_song_RDD.join(song_titles).join(songs_rating_counts_RDD)

In [321]:
user_recommendations_rating_title_and_count_RDD.take(1)

[(432133, ((-0.00011697554938471183, u'Subbacultcha: Pixies-1987'), 1))]

In [322]:
user_recommendations_rating_title_and_count_RDD = \
    user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

In [323]:
user_recommendations_rating_title_and_count_RDD.take(1)

[(u'Subbacultcha: Pixies-1987', -0.00011697554938471183, 1)]

In [324]:
top_songs = user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=50).distinct().takeOrdered(5, key=lambda x: -x[1])

print ('TOP recommended songs (with more than 50 listen count):\n%s' %
        '\n'.join(map(str, top_songs)))

TOP recommended songs (with more than 50 listen count):
(u'"I Kissed A Girl": "Katy Perry"-2008', 0.007224891936197882, 101)
(u'"Catch You Baby (Steve Pitron & Max Sanna Radio Edit)": "Lonnie Gordon"-0', 0.00702203739380804, 403)
(u'"Gives You Hell": "The All-American Rejects"-2008', 0.0064229740825680805, 61)
(u'Toxic: "Britney Spears"-2003', 0.006368472895766891, 66)
(u'"Half Of My Heart": "John Mayer"-0', 0.006260219596973631, 97)


In [340]:
import difflib
import random
import requests
import ipywidgets as widgets
from ipywidgets import Button, Layout
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Text
from IPython.display import clear_output, display
import time
import datetime

user_list = pd.read_csv('file:/vagrant/songProfiles.csv')
user_list['songs'] = user_list['songs'].str.split('|')
# Button for generating the recommendations

def on_rec_clicked(rec):
    clear_output()
    print('Loading Recommendations: ............')
    st_time = datetime.datetime.now()
    user=user_list['user_id'][tab.selected_index]
    user_id = user_mapping.filter(lambda x: x[0] == user ).map(lambda x: x[1]).take(1)[0]
    user_id_songs_listened = song_data_final.filter(lambda x: x[0] == user_id)
    songs_listened_by_user = user_id_songs_listened.map(lambda x: x[1])
    songs_listened_by_user_1=[x for x in songs_listened_by_user.toLocalIterator()]
    user_not_listened_songs_RDD = song_mapping_2.filter(lambda x: x[0] not in songs_listened_by_user_1)\
    .map(lambda x: (user_id, x[0]))
    
    user_recommendations_RDD = model.predictAll(user_not_listened_songs_RDD)
    user_recommendations_song_RDD = user_recommendations_RDD.map(lambda x: (x.product, x.rating))
    song_titles = song_mapping_2.map(lambda x: (int(x[0]),x[1] + ': ' + x[3]))
    top_songs = \
        user_recommendations_song_RDD.join(song_titles).join(songs_rating_counts_RDD)\
        .map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))\
        .filter(lambda r: r[2]>=50).distinct().takeOrdered(10, key=lambda x: -x[1])
    print('Recommendation took {} '.format(datetime.datetime.now() - st_time))
    display(pd.DataFrame({'songs':[row[0] for row in top_songs]}))
    #results = is_model.get_user_items(child.description)
    #clear_output()
    #display(results[['rank','song']])

rec = Button(description='Generate  Recommendations',
       layout=Layout(width='100%', height='80px'), button_style = 'info')

rec.on_click(on_rec_clicked)
    
#Clear Button
clear = Button(description='Clear',
           layout=Layout(width='100%', height='80px'),button_style = 'primary')

def on_clear(clear):
    clear_output()
    return(tab)

clear.on_click(on_clear)


tab_list = user_list['user_name']
children = []
tab = widgets.Tab()
for i in range(len(tab_list)):
    #tab.children = (dropdown,button_form)
    tab.set_title(i, str(tab_list[i]))
    songs = user_list['songs'][i]
    user_song_labels= [widgets.Label(str(song)) for song in songs]
    child = widgets.VBox(user_song_labels,description = user_list['user_id'][i],
            layout=Layout(
               display='flex',
             flex_flow='column',
               border='solid 2px',
               width='75%'))

    button_form = Box(children=(rec,clear),
                    layout=Layout(display='flex',
                     flex_flow='column',
                    border='solid 2px',
                    fill='grey',          
           align_items='center',
           width='50%'))
    full_form = widgets.HBox([child,button_form])
    children.append(full_form)
    


tab.children = tuple(children)
tab

Loading Recommendations: ............
